In [3]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
import clickhouse_connect

load_dotenv('/home/jovyan/.env')

# Чтение параметров подключения
CLICKHOUSE_HOST = os.getenv('CLICKHOUSE_HOST')
CLICKHOUSE_PORT = int(os.getenv('CLICKHOUSE_PORT'))
CLICKHOUSE_DB = os.getenv('CLICKHOUSE_DB')
CLICKHOUSE_USER = os.getenv('CLICKHOUSE_USER')
CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD')

try:
    client = clickhouse_connect.get_client(
        host=CLICKHOUSE_HOST,
        port=CLICKHOUSE_PORT,
        database=CLICKHOUSE_DB,
        username=CLICKHOUSE_USER,
        password=CLICKHOUSE_PASSWORD
    )
    print("Подключение к ClickHouse установлено!")
except Exception as e:
    print(f"Ошибка подключения: {str(e)}")

Подключение к ClickHouse установлено!


In [7]:
# Загрузка logins.csv
def load_logins():
    logins_csv_path = "/home/jovyan/data/logins.csv"
    exists_logins = bool(client.query("SELECT 1 FROM logins LIMIT 1").result_rows)
    if exists_logins:
        return "Данные уже есть в базе."
    if not os.path.isfile(logins_csv_path):
        return f"Файл не найден: {logins_csv_path}"
    else:
        try:
            required_columns = ('Timestamp', 'GUID', 'OuterIP', 'InnerIP', 'OpenVPNServer', 'NgToken')
            logins_df = pd.read_csv(logins_csv_path, usecols=required_columns)
            if not logins_df.empty:
                logins_df['Timestamp'] = pd.to_datetime(logins_df['Timestamp'])
                client.insert(
                    'logins',
                    logins_df,
                )
                return f"Загружено {len(logins_df)} записей в таблицу logins."
            else:
                return "В файле нет данных для загрузки."
        except Exception as e:
            return f"Ошибка при загрузке logins.csv: {e}"
result_load_logins = load_logins()
print(result_load_logins)

Данные уже есть в базе.


In [6]:
# Загрузка user_info.json
def load_users():
    user_info_path = "/home/jovyan/data/user_info.json"
    exists_user = bool(client.query("SELECT 1 FROM user_info LIMIT 1").result_rows)
    if exists_user:
        return "Данные уже есть в базе."
    if not os.path.isfile(user_info_path):
        return f"Файл не найден: {user_info_path}"
    else:
            with open(user_info_path, 'r', encoding='utf-8') as f:
                user_info_data = json.load(f)
            df = pd.DataFrame(user_info_data)
            if not df.empty:
                 try:
                    client.insert(
                        'user_info',
                        df
                    )
                    return f"✅ Загружено {len(user_info_data)} записей в таблицу user_info."
                 except  Exception as e:
                   return f"Ошибка при загрузке user_info.json: {e}"
            else:
                return "В файле нет данных для загрузки."
result_load_users = load_users()
print(result_load_users)

Данные уже есть в базе.
